# Cell 1: Setup and Installation

In [ ]:
!pip install librosa scikit-learn matplotlib numpy pandas noisereduce tensorflow shap

# Cell 1.5

In [ ]:
# Cell 1.5: Updated - No unzip needed since files are already extracted
import os
import shutil

# Paths to your uploaded files (they're already extracted)
dataset_path = '/kaggle/input/train-audio'
data_dir = '/kaggle/working/violin-emotion-analysis/data'
csv_path = '/kaggle/input/train-audio/emotion_labels.csv'

# Create the working directory
os.makedirs(data_dir, exist_ok=True)

# Copy all audio files from the dataset to our working directory
audio_source_dir = '/kaggle/input/train-audio/audio'
if os.path.exists(audio_source_dir):
    # Copy all WAV files to our working directory
    for file_name in os.listdir(audio_source_dir):
        if file_name.endswith('.wav'):
            source_path = os.path.join(audio_source_dir, file_name)
            dest_path = os.path.join(data_dir, file_name)
            shutil.copy2(source_path, dest_path)
    
    print(f"✅ Copied audio files to working directory")
else:
    print("❌ Audio folder not found!")

# Check folder structure
print("\nData directory structure:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

# Check if CSV exists
if not os.path.exists(csv_path):
    print("\n❌ WARNING: 'emotion_labels.csv' not found!")
else:
    print(f"\n✅ CSV file found: {csv_path}")
    # Also copy CSV to working directory for consistency
    shutil.copy2(csv_path, '/kaggle/working/violin-emotion-analysis/emotion_labels.csv')
    print("✅ CSV copied to working directory")

print(f"\n📊 Total audio files ready for processing: {len([f for f in os.listdir(data_dir) if f.endswith('.wav')])}")

# Cell 2: Imports and Updated Utility Functions

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Attention, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import noisereduce as nr
import shap
import joblib
import warnings
warnings.filterwarnings('ignore')

# Create directories
os.makedirs('/kaggle/working/violin-emotion-analysis/data', exist_ok=True)
os.makedirs('/kaggle/working/violin-emotion-analysis/models', exist_ok=True)

def extract_temporal_features(file_path, sr=44100, frame_length=2048, hop_length=512, window_size=3.0, hop_time=1.5):
    """
    Extract time-based audio features from overlapping windows (captures emotion dynamics)
    """
    y, sr = librosa.load(file_path, sr=sr)
    y = nr.reduce_noise(y=y, sr=sr)

    total_duration = librosa.get_duration(y=y, sr=sr)
    step = int(hop_time * sr)
    window = int(window_size * sr)

    feature_sequences = []
    for start in range(0, len(y) - window, step):
        segment = y[start:start + window]
        segment_features = []
        mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13)
        chroma = librosa.feature.chroma_stft(y=segment, sr=sr)
        spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sr)
        rms = librosa.feature.rms(y=segment)
        zcr = librosa.feature.zero_crossing_rate(y=segment)
        centroid = librosa.feature.spectral_centroid(y=segment, sr=sr)
        bandwidth = librosa.feature.spectral_bandwidth(y=segment, sr=sr)
        
        # Combine flattened statistics
        feature_vec = np.hstack([
            np.mean(mfccs, axis=1),
            np.std(mfccs, axis=1),
            np.mean(chroma, axis=1),
            np.mean(spectral_contrast, axis=1),
            np.mean(rms),
            np.mean(zcr),
            np.mean(centroid),
            np.mean(bandwidth)
        ])
        feature_sequences.append(feature_vec)

    return np.array(feature_sequences)

def create_temporal_dataset(data_dir, emotion_csv_path):
    """
    Create dataset with temporal features and soft emotion labels.
    emotion_csv_path must include columns: filename, emotion, and soft labels like happy, sad, calm, etc.
    """
    annotations = pd.read_csv(emotion_csv_path)
    X, y_soft = [], []

    for _, row in annotations.iterrows():
        file_path = os.path.join(data_dir, row['filename'])
        if not os.path.exists(file_path): 
            continue

        features = extract_temporal_features(file_path)
        X.append(features)
        y_soft.append(row[4:].values.astype(float))  # ← NEW (gives 3 dimensions)
        
    # Pad sequences for LSTM
    max_len = max(x.shape[0] for x in X)
    num_features = X[0].shape[1]
    X_padded = np.zeros((len(X), max_len, num_features))
    for i, seq in enumerate(X):
        X_padded[i, :seq.shape[0], :] = seq

    return np.array(X_padded), np.array(y_soft)


# Cell 3: Data Preparation

In [ ]:
data_dir = '/kaggle/working/violin-emotion-analysis/data'
emotion_csv_path = '/kaggle/working/violin-emotion-analysis/emotion_labels.csv'  # <-- upload your CSV here

if not os.path.exists(emotion_csv_path):
    print("Please upload 'emotion_labels.csv' with soft emotion probabilities.")
else:
    X, y_soft = create_temporal_dataset(data_dir, emotion_csv_path)
    print(f"Loaded {X.shape[0]} samples with {X.shape[2]} features each and {y_soft.shape[1]} soft emotion dimensions.")

# Cell 4: Model Training (LSTM + Random Forest)

In [ ]:
# Cell 4: Model Training (LSTM + RandomForest) - FIXED VERSION
from tensorflow.keras.models import Model

# Split data
X_train, X_test, y_train_soft, y_test_soft = train_test_split(
    X, y_soft, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Emotion dimensions: {y_train_soft.shape[1]}")

# Normalize feature values
scaler = StandardScaler()
for i in range(X_train.shape[0]):
    X_train[i] = scaler.fit_transform(X_train[i])
for i in range(X_test.shape[0]):
    X_test[i] = scaler.transform(X_test[i])

joblib.dump(scaler, '/kaggle/working/violin-emotion-analysis/models/scaler.pkl')

# Define LSTM model with Attention layer
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = LSTM(128, return_sequences=True)(input_layer)
x = Attention()([x, x])
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = LSTM(64)(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(y_train_soft.shape[1], activation='softmax')(x)
lstm_model = Model(inputs=input_layer, outputs=output_layer)

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)

print("Training LSTM model with temporal and soft label support...")
history = lstm_model.fit(
    X_train, y_train_soft,
    validation_data=(X_test, y_test_soft),
    epochs=80, batch_size=16,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Train RandomForest on averaged features (feature-based model)
X_train_flat = np.mean(X_train, axis=1)
X_test_flat = np.mean(X_test, axis=1)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flat, np.argmax(y_train_soft, axis=1))

# FIXED PREDICTION SECTION WITH DEBUG INFO
print("\n" + "="*50)
print("PREDICTION DEBUG INFO:")
print("="*50)

# LSTM predictions
lstm_probs = lstm_model.predict(X_test)
print(f"LSTM probabilities shape: {lstm_probs.shape}")
print(f"LSTM probs sample: {lstm_probs[0]}")

# Random Forest predictions
rf_probs = rf_model.predict_proba(X_test_flat)
print(f"RF probabilities shape: {rf_probs.shape}")
print(f"RF probs sample: {rf_probs[0]}")

# Ensure both probability arrays have the same dimensions
if rf_probs.shape[1] < lstm_probs.shape[1]:
    # Pad RF probabilities if needed
    rf_probs_padded = np.zeros((rf_probs.shape[0], lstm_probs.shape[1]))
    rf_probs_padded[:, :rf_probs.shape[1]] = rf_probs
    rf_probs = rf_probs_padded
    print(f"Padded RF probabilities to: {rf_probs.shape}")

# Hybrid fusion
hybrid_probs = 0.6 * lstm_probs + 0.4 * rf_probs
print(f"Hybrid probs shape: {hybrid_probs.shape}")
print(f"Hybrid probs sample: {hybrid_probs[0]}")

# Get predictions
hybrid_pred = np.argmax(hybrid_probs, axis=1)
true_labels = np.argmax(y_test_soft, axis=1)

print(f"True labels: {true_labels}")
print(f"Predicted labels: {hybrid_pred}")

# Calculate accuracies
accuracy = accuracy_score(true_labels, hybrid_pred)

# Individual model accuracies for comparison
lstm_pred = np.argmax(lstm_probs, axis=1)
rf_pred = rf_model.predict(X_test_flat)

lstm_accuracy = accuracy_score(true_labels, lstm_pred)
rf_accuracy = accuracy_score(true_labels, rf_pred)

print("\n" + "="*50)
print("RESULTS:")
print("="*50)
print(f"LSTM Model Accuracy: {lstm_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"Hybrid Model Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nDetailed Classification Report (Hybrid Model):")
print(classification_report(true_labels, hybrid_pred, target_names=['Happy', 'Sad', 'Angry']))

# Save models
lstm_model.save('/kaggle/working/violin-emotion-analysis/models/lstm_hybrid_model.h5')
joblib.dump(rf_model, '/kaggle/working/violin-emotion-analysis/models/rf_model.pkl')
print("\n✅ Models saved successfully!")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.show()

# Cell 5: Explainability and Visualization

In [ ]:
# Cell 5: Enhanced SHAP Analysis with Emotion-Specific Features
print("🎵 SHAP Analysis - Feature Importance by Emotion")

# Get emotion labels from your CSV
emotion_labels = ['Happy', 'Sad', 'Angry']

# Create SHAP explainer
explainer = shap.Explainer(rf_model, X_train_flat[:100])  # Use first 100 training samples
shap_values = explainer(X_test_flat[:50])

print("🔍 Analyzing feature importance for each emotion...")

# Create separate plots for each emotion
for emotion_idx, emotion_name in enumerate(emotion_labels):
    print(f"\n🎻 {emotion_name.upper()} EMOTION:")
    print("="*40)
    
    # Get SHAP values for this specific emotion
    emotion_shap_values = shap_values.values[:, :, emotion_idx]
    
    # Get top 10 most important features for this emotion
    feature_importance = np.mean(np.abs(emotion_shap_values), axis=0)
    top_feature_indices = np.argsort(feature_importance)[-10:][::-1]  # Top 10
    
    print(f"Top features that predict {emotion_name}:")
    for i, idx in enumerate(top_feature_indices):
        importance = feature_importance[idx]
        print(f"  {i+1}. Feature {idx}: {importance:.4f}")
    
    # Create feature names based on what we know about the extraction
    feature_categories = []
    for i in range(49):  # You have 49 features
        if i < 26:  # First 26 are MFCCs (13 mean + 13 std)
            if i < 13:
                feature_categories.append(f"MFCC_{i+1}_mean")
            else:
                feature_categories.append(f"MFCC_{i-12}_std")
        elif i < 38:  # Next 12 are Chroma (mean only)
            feature_categories.append(f"Chroma_{i-25}")
        elif i < 45:  # Next 7 are Spectral Contrast
            feature_categories.append(f"Spectral_Contrast_{i-37}")
        elif i == 45:
            feature_categories.append("RMS_energy")
        elif i == 46:
            feature_categories.append("Zero_Crossing_Rate")
        elif i == 47:
            feature_categories.append("Spectral_Centroid")
        elif i == 48:
            feature_categories.append("Spectral_Bandwidth")
        else:
            feature_categories.append(f"Feature_{i}")
    
    print(f"\n📊 Named features for {emotion_name}:")
    for i, idx in enumerate(top_feature_indices[:5]):  # Top 5 with names
        feature_name = feature_categories[idx]
        importance = feature_importance[idx]
        print(f"  {i+1}. {feature_name}: {importance:.4f}")

# Create the main SHAP summary plot
print("\n📈 Generating main SHAP summary plot...")
shap.summary_plot(shap_values, X_test_flat[:50], feature_names=feature_categories, show=False)
plt.title("SHAP Feature Importance - All Emotions")
plt.tight_layout()
plt.show()

# Additional: Force plot for a single prediction to see detailed reasoning
print("\n🔬 Detailed prediction breakdown for first test sample:")
shap.force_plot(explainer.expected_value[0], shap_values.values[0,:,0], X_test_flat[0,:], 
                feature_names=feature_categories, matplotlib=True, show=False)
plt.title(f"Prediction Breakdown: {emotion_labels[np.argmax(y_test_soft[0])]} → {emotion_labels[rf_model.predict(X_test_flat[0:1])[0]]}")
plt.tight_layout()
plt.show()

# Cell 6: Real-Time Prediction Simulation on New Audio Files

In [ ]:
new_audio_dir = '/kaggle/working/violin-emotion-analysis/new_audio'  # put new audios here
emotion_labels = pd.read_csv(emotion_csv_path).columns[3:].tolist()  # same soft labels as training

new_files = [f for f in os.listdir(new_audio_dir) if f.endswith('.wav')]
print(f"Found {len(new_files)} new audio files for prediction.")

for f in new_files:
    path = os.path.join(new_audio_dir, f)
    
    # Extract features
    features = extract_temporal_features(path)
    
    # Pad sequence to match LSTM input
    padded = np.zeros((1, X_train.shape[1], X_train.shape[2]))
    padded[0, :features.shape[0], :] = features
    
    # LSTM prediction
    lstm_probs = lstm_model.predict(padded)
    
    # RF prediction
    rf_probs = rf_model.predict_proba(np.mean(padded, axis=1))
    rf_probs = np.array([np.pad(p, (0, len(emotion_labels) - len(p))) for p in rf_probs])
    
    # Hybrid fusion
    hybrid_probs = 0.6*lstm_probs + 0.4*rf_probs
    
    # Perceptual smoothing
    smoothed = np.convolve(np.mean(hybrid_probs, axis=0), np.ones(3)/3, mode='same')
    
    # Plot results
    plt.figure(figsize=(8, 5))
    plt.bar(emotion_labels, smoothed)
    plt.title(f"Predicted Emotions for {f}")
    plt.ylabel("Probability")
    plt.ylim(0,1)
    plt.show()
    
    # Print prediction
    pred_emotion = emotion_labels[np.argmax(smoothed)]
    print(f"Audio: {f} → Predicted Emotion: {pred_emotion}")
    print("Probability per emotion:")
    for label, prob in zip(emotion_labels, smoothed):
        print(f"  {label}: {prob:.3f}")
    print("\n" + "-"*40 + "\n")

